# Query

In [1]:
from wanglabconsts import OUTCOME_DEATH, OUTCOME_EDEMA, ENCOUNTERS_FILE, PREDICT_OFFSET, OUTCOME_DEATH, QUERIED_DIR

In [2]:
from wanglabquerying import main

2022-08-24 14:53:40,290 INFO cyclops.orm     - Database setup, ready to run queries!


In [3]:
import pandas as pd

In [4]:
import time

import numpy as np

from cyclops.processors.column_names import (
    ADMIT_TIMESTAMP,
    DIAGNOSIS_CODE,
    DIAGNOSIS_TRAJECTORY,
    DISCHARGE_TIMESTAMP,
    ENCOUNTER_ID,
    EVENT_NAME,
    EVENT_TIMESTAMP,
    EVENT_VALUE,
    EVENT_VALUE_UNIT,
    TIMESTEP,
    YEAR,
)
from cyclops.processors.constants import (
    BINARY,
    CATEGORICAL_INDICATOR,
    FEATURE_INDICATOR_ATTR,
    FEATURE_MAPPING_ATTR,
    FEATURE_TYPE_ATTR,
    FEATURE_TYPES,
    FEATURES,
    MEAN,
    MIN_MAX,
    MISSING_CATEGORY,
    NUMERIC,
    ORDINAL,
    STRING,
    TARGETS,
)
from cyclops.processors.feature.split import intersect_datasets
from cyclops.utils.file import load_dataframe, save_dataframe

In [5]:
from cyclops.processors.cleaning import (
    normalize_categories,
    normalize_names,
    normalize_values,
)
from cyclops.utils.file import join

In [6]:
t = time.time()
cohort, events = main()
print(time.time() - t)
cohort

2022-08-24 14:53:44,446 INFO cyclops.orm     - Query returned successfully!
2022-08-24 14:53:44,447 INFO cyclops.utils.profile - Finished executing function run_query in 4.078502 s
2022-08-24 14:53:46,112 INFO cyclops.orm     - Query returned successfully!
2022-08-24 14:53:46,113 INFO cyclops.utils.profile - Finished executing function run_query in 1.665209 s
2022-08-24 14:53:48,831 INFO cyclops.orm     - Query returned successfully!
2022-08-24 14:53:48,833 INFO cyclops.utils.profile - Finished executing function run_query in 2.205238 s
2022-08-24 14:53:54,604 INFO cyclops.orm     - Query returned successfully!
2022-08-24 14:53:54,605 INFO cyclops.utils.profile - Finished executing function run_query in 1.808606 s
2022-08-24 14:53:55,117 INFO cyclops.orm     - Query returned successfully!
2022-08-24 14:53:55,119 INFO cyclops.utils.profile - Finished executing function run_query in 0.183559 s
2022-08-24 14:54:33,154 INFO cyclops.orm     - Query returned successfully!
2022-08-24 14:54:33

819.357593536377


,encounter_id,admit_timestamp,discharge_timestamp,age,sex,hospital_id,outcome_death,readmission,from_nursing_home_mapped,from_acute_care_institution_mapped,...,X-ray_shoulder,CT_shoulder,MRI_shoulder,Ultrasound_shoulder,X-ray_whole_body,CT_whole_body,MRI_whole_body,Ultrasound_whole_body,ip_charlson_derived,outcome_edema
0,15535918,2016-08-30 12:00:00,2016-09-09 18:09:00,28,F,THPC,False,nota,False,False,...,0,0,0,0,0,0,0,0,0,False
1,15612180,2014-11-12 04:11:00,2014-11-20 18:53:00,65,F,THPC,False,nota,False,False,...,0,0,0,0,2,0,0,0,0,False
2,15682885,2017-04-06 05:15:00,2017-04-07 22:06:00,73,F,THPC,False,new_to_acute,False,False,...,0,0,0,0,1,1,1,0,0,False
3,15712185,2012-11-14 03:30:00,2012-11-26 14:42:00,65,M,THPC,False,new_to_acute,False,False,...,0,0,0,0,2,0,1,0,0,False
4,15913447,2017-12-18 21:00:00,2017-12-27 16:34:00,79,M,THPC,False,nota,False,False,...,0,0,0,0,0,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113589,15987592,2019-03-18 15:02:00,2019-03-18 16:32:00,62,F,THPM,True,new_to_acute,False,False,...,0,0,0,0,0,0,0,0,6,False
113590,12779266,2019-07-24 15:49:00,2019-07-25 11:08:00,75,F,SBK,True,planned_from_acute,False,False,...,0,0,0,0,0,0,0,0,6,False
113591,14167874,2020-01-20 19:55:00,2020-01-31 14:43:00,54,M,MSH,True,nota,False,False,...,0,0,0,0,0,0,0,0,6,False
113592,13933236,2018-12-29 20:53:00,2019-01-21 13:00:00,57,M,UHNTG,True,nota,False,False,...,0,0,0,0,2,0,1,0,6,False


In [7]:
cohort[OUTCOME_DEATH].sum() / len(cohort)

0.10195080726094688

In [8]:
cohort[OUTCOME_EDEMA].sum() / len(cohort)

0.03752839058400972

In [9]:
# Intersect over encounter IDs to get only those encounters common to both
# cohort, events = intersect_datasets([cohort, events], ENCOUNTER_ID)

In [10]:
save_dataframe(cohort, ENCOUNTERS_FILE)

2022-08-24 15:07:19,871 INFO cyclops.utils.file - Saving dataframe to ./wanglab_data/encounters.parquet


'./wanglab_data/encounters.parquet'

# Clean / Preprocess

In [11]:
events[EVENT_NAME] = normalize_names(events[EVENT_NAME])
#events[EVENT_VALUE] = normalize_values(events[EVENT_VALUE])

In [12]:
# Create the target as a timeseries event
death_events = cohort[cohort[OUTCOME_DEATH] == True]
death_events = death_events[[ENCOUNTER_ID, DISCHARGE_TIMESTAMP]]

# Offset death time - i.e., should predict death time in advance
death_events[DISCHARGE_TIMESTAMP] = death_events[DISCHARGE_TIMESTAMP] - pd.DateOffset(
    hours=PREDICT_OFFSET
)

death_events = death_events.rename({DISCHARGE_TIMESTAMP: EVENT_TIMESTAMP}, axis=1)
death_events[EVENT_NAME] = OUTCOME_DEATH
death_events[EVENT_VALUE] = 1
death_events.head(5)

,encounter_id,event_timestamp,event_name,event_value
95316,15110365,2016-02-01 13:50:00,outcome_death,1
95317,15190337,2015-06-12 19:30:00,outcome_death,1
95318,15202353,2018-02-07 10:30:00,outcome_death,1
95319,15244933,2017-06-14 19:20:00,outcome_death,1
95320,15247661,2018-01-27 15:40:00,outcome_death,1


In [13]:
events = pd.concat([events, death_events])

In [14]:
#save_dataframe(events, join(QUERIED_DIR, "batch_0000.parquet"))